In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [2]:
raw_data_folder = os.path.join(os.getcwd(),os.pardir, 'data','raw_data')
df_raw = pd.read_csv(os.path.join(raw_data_folder, 'turnstile_180721.txt'))

First, we'll take a look at how the raw data files are structured.

In [69]:
df_raw.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,next_interval
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/14/2018,00:00:00,REGULAR,6690808,2267742,00:00:00-04:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/14/2018,04:00:00,REGULAR,6690813,2267743,04:00:00-08:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/14/2018,08:00:00,REGULAR,6690819,2267744,08:00:00-12:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/14/2018,12:00:00,REGULAR,6690823,2267744,12:00:00-16:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/14/2018,16:00:00,REGULAR,6690828,2267744,16:00:00-20:00:00


The data is organized by station and turnstile along with entries and exits at what seems like specific intervals. The entries and exits values appear to be values from a meter that keeps accumulating. Which means that from 7/14/2018 between 00:00:00 and 04:00:00 there were 5 people who entered through that specific turnstile at 59th street station. 

In [4]:
df_raw.groupby(['DESC']).size().sort_values(ascending=False).reset_index(name='Count')

,DESC,Count
0,REGULAR,195112
1,RECOVR AUD,837


Let's see if there are duplicate data at a given turnstile and timestamp.

In [5]:
df_raw.groupby(['C/A', 'UNIT', 'SCP', 'DATE', 'TIME']).size().sort_values(ascending=False).reset_index(name='Count').head()

,C/A,UNIT,SCP,DATE,TIME,Count
0,TRAM2,R469,00-05-01,07/20/2018,21:00:00,1
1,N139,R355,00-00-01,07/20/2018,09:00:00,1
2,N139,R355,00-00-02,07/14/2018,21:00:00,1
3,N139,R355,00-00-02,07/14/2018,17:00:00,1
4,N139,R355,00-00-02,07/14/2018,13:00:00,1


In this raw data file it doesn't seem like there are duplicate entries. But in the cleaning up process it probably will be prudent to take the mean to deal with any duplicates that occurs.

In [6]:
df_raw.groupby(['TIME', 'DESC']).size().sort_values(ascending=False).reset_index(name='Count')

,TIME,DESC,Count
0,20:00:00,REGULAR,16933
1,16:00:00,REGULAR,16908
2,04:00:00,REGULAR,16903
3,00:00:00,REGULAR,16900
4,08:00:00,REGULAR,16898
5,12:00:00,REGULAR,16858
6,01:00:00,REGULAR,11909
7,05:00:00,REGULAR,11901
8,21:00:00,REGULAR,11897
9,17:00:00,REGULAR,11883


In [7]:
df_raw['next_interval'] = df_raw.groupby(['C/A', 'UNIT', 'SCP', 'DATE'])['TIME'].transform(lambda x: x + '-' + x.shift(-1))
df_raw.groupby(['next_interval']).size().sort_values(ascending=False)

next_interval
00:00:00-04:00:00    16952
16:00:00-20:00:00    16949
08:00:00-12:00:00    16896
04:00:00-08:00:00    16889
12:00:00-16:00:00    16874
01:00:00-05:00:00    11933
09:00:00-13:00:00    11896
05:00:00-09:00:00    11893
17:00:00-21:00:00    11893
13:00:00-17:00:00    11882
18:00:00-22:00:00      734
14:00:00-18:00:00      733
10:00:00-14:00:00      731
02:00:00-06:00:00      727
06:00:00-10:00:00      720
19:00:00-23:00:00      273
15:00:00-19:00:00      273
11:00:00-15:00:00      273
03:00:00-07:00:00      273
07:00:00-11:00:00      273
08:22:00-12:22:00      259
04:22:00-08:22:00      259
12:22:00-16:22:00      259
00:22:00-04:22:00      259
16:22:00-20:22:00      259
14:30:00-18:30:00      140
10:30:00-14:30:00      140
06:30:00-10:30:00      140
02:30:00-06:30:00      140
18:30:00-22:30:00      140
                     ...  
12:50:11-17:02:11        1
12:50:10-17:02:10        1
12:50:09-17:02:09        1
12:49:59-17:01:59        1
12:51:08-17:03:08        1
12:49:45-17:01

The most commonly occuring interval of reads seems to be every 4 hours. The other thing to note here is that there are missing datapoints. Take the sets of turnstiles reporting at 00:00:00, 04:00:00, up to 20:00:00. The 00:00:00-04:00:00, 04:00:00-08:00:00 should all have the exact same amount of counts. Since it doesn't, it is very likely that data is missing. 

In addition, since the files are broken out, week by week, the very last readings from each of the files needs to be passed onto the following week's data.  Taking this into account, the goal of the data cleaning script is to aggregate the amount of people entering and exiting the station between the _regular_ intervals to make it easier for further analysis. The goal is to get the data into this format.

| Station | Date | Interval | Entries | Exits |
| --- | --- | --- | --- | --- |
| Station A | 7/15/2018 | 00:00:00-04:00:00 | 100 | 50 |

I've written a separate R script that will help clean-up the data which is located in the __[src folder](../src/)__

Missing data is _not_ imputed in the script.

The website seems to indicate two different headers so let's check some of the older files.

In [8]:
df_raw2 = pd.read_csv(os.path.join(raw_data_folder, 'turnstile_140705.txt'), header=None)
df_raw2.iloc[0:10]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42
0,A002,R051,02-00-00,06-28-14,00:00:00,REGULAR,4679542,1591173,06-28-14,04:00:00,REGULAR,4679583.0,1591180.0,06-28-14,08:00:00,REGULAR,4679603.0,1591196.0,06-28-14,12:00:00,REGULAR,4679707.0,1591296.0,06-28-14,16:00:00,REGULAR,4680003.0,1591342.0,06-28-14,20:00:00,REGULAR,4680439.0,1591378.0,06-29-14,00:00:00,REGULAR,4680619.0,1591398.0,06-29-14,04:00:00,REGULAR,4680669.0,1591403.0
1,A002,R051,02-00-00,06-29-14,08:00:00,REGULAR,4680696,1591404,06-29-14,12:00:00,REGULAR,4680784.0,1591424.0,06-29-14,16:00:00,REGULAR,4681039.0,1591454.0,06-29-14,20:00:00,REGULAR,4681350.0,1591491.0,06-30-14,00:00:00,REGULAR,4681458.0,1591527.0,06-30-14,04:00:00,REGULAR,4681474.0,1591529.0,06-30-14,08:00:00,REGULAR,4681509.0,1591619.0,06-30-14,12:00:00,REGULAR,4681670.0,1591876.0
2,A002,R051,02-00-00,06-30-14,16:00:00,REGULAR,4681988,1591918,06-30-14,20:00:00,REGULAR,4682886.0,1591985.0,07-01-14,00:00:00,REGULAR,4683127.0,1592009.0,07-01-14,04:00:00,REGULAR,4683144.0,1592012.0,07-01-14,08:00:00,REGULAR,4683178.0,1592110.0,07-01-14,09:18:46,DOOR OPEN,4683246.0,1592239.0,07-01-14,09:18:51,DOOR OPEN,4683246.0,1592239.0,07-01-14,09:44:28,LOGON,4683246.0,1592239.0
3,A002,R051,02-00-00,07-01-14,09:44:40,DOOR CLOSE,4683246,1592239,07-01-14,09:44:41,DOOR OPEN,4683246.0,1592239.0,07-01-14,09:46:25,DOOR CLOSE,4683246.0,1592239.0,07-01-14,09:46:26,DOOR OPEN,4683246.0,1592239.0,07-01-14,09:47:53,LGF-MAN,4683246.0,1592239.0,07-01-14,09:50:34,DOOR CLOSE,4683247.0,1592242.0,07-01-14,10:16:13,DOOR OPEN,4683256.0,1592279.0,07-01-14,10:17:31,DOOR CLOSE,4683256.0,1592279.0
4,A002,R051,02-00-00,07-01-14,12:00:00,REGULAR,4683339,1592331,07-01-14,16:00:00,REGULAR,4683660.0,1592393.0,07-01-14,20:00:00,REGULAR,4684497.0,1592455.0,07-02-14,00:00:00,REGULAR,4684723.0,1592480.0,07-02-14,04:00:00,REGULAR,4684747.0,1592485.0,07-02-14,08:00:00,REGULAR,4684783.0,1592566.0,07-02-14,12:00:00,REGULAR,4684941.0,1592818.0,07-02-14,16:00:00,REGULAR,4685264.0,1592879.0
5,A002,R051,02-00-00,07-02-14,20:00:00,REGULAR,4686127,1592954,07-03-14,00:00:00,REGULAR,4686341.0,1592980.0,07-03-14,04:00:00,REGULAR,4686369.0,1592982.0,07-03-14,08:00:00,REGULAR,4686404.0,1593063.0,07-03-14,08:16:40,REGULAR,4686416.0,1593078.0,07-03-14,12:00:00,REGULAR,4686573.0,1593283.0,07-03-14,16:00:00,REGULAR,4687123.0,1593335.0,07-03-14,20:00:00,REGULAR,4687802.0,1593389.0
6,A002,R051,02-00-00,07-04-14,00:00:00,REGULAR,4688005,1593414,07-04-14,04:00:00,REGULAR,4688033.0,1593415.0,07-04-14,08:00:00,REGULAR,4688045.0,1593437.0,07-04-14,12:00:00,REGULAR,4688110.0,1593499.0,07-04-14,16:00:00,REGULAR,4688276.0,1593540.0,07-04-14,20:00:00,REGULAR,4688522.0,1593573.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,A002,R051,02-00-01,06-28-14,00:00:00,REGULAR,4366970,952155,06-28-14,04:00:00,REGULAR,4366997.0,952156.0,06-28-14,08:00:00,REGULAR,4367018.0,952169.0,06-28-14,12:00:00,REGULAR,4367143.0,952225.0,06-28-14,16:00:00,REGULAR,4367391.0,952277.0,06-28-14,20:00:00,REGULAR,4367742.0,952308.0,06-29-14,00:00:00,REGULAR,4367889.0,952330.0,06-29-14,04:00:00,REGULAR,4367924.0,952331.0
8,A002,R051,02-00-01,06-29-14,08:00:00,REGULAR,4367933,952334,06-29-14,12:00:00,REGULAR,4367990.0,952347.0,06-29-14,16:00:00,REGULAR,4368163.0,952382.0,06-29-14,20:00:00,REGULAR,4368392.0,952412.0,06-30-14,00:00:00,REGULAR,4368480.0,952421.0,06-30-14,04:00:00,REGULAR,4368490.0,952422.0,06-30-14,08:00:00,REGULAR,4368516.0,952460.0,06-30-14,12:00:00,REGULAR,4368690.0,952589.0
9,A002,R051,02-00-01,06-30-14,16:00:00,REGULAR,4368964,952642,06-30-14,20:00:00,REGULAR,4369759.0,952699.0,07-01-14,00:00:00,REGULAR,4369957.0,952717.0,07-01-14,04:00:00,REGULAR,4369967.0,952725.0,07-01-14,08:00:00,REGULAR,4370003.0,952768.0,07-01-14,09:28:16,DOOR OPEN,4370082.0,952849.0,07-01-14,09:28:31,DOOR CLOSE,4370082.0,952849.0,07-01-14,09:28:32,DOOR OPEN,4370082.0,952849.0


This is quite the structure. The data seems to be organized in a way such that the first 3 columns identifies a specific turnstile. Then the next 8 sets of 5 columns, i.e. columns 3-7, 8-12..., corresponds to the date, time, type of read, entries and exits. On top of that, every 7 row, there is missing data from columns 34-42. The R script mentioned above will take this data format into account and clean it up. After running the R script the cleaned up data will be located in the __[processed](../data/processed)__ folder.

Well let's see how this data file compares to the _newer_ ones.

In [82]:
columns_base = list(range(0,8))
df = df_raw2[columns_base]

col_list = [list(range(x,x+5)) for x in range(8,43,5)]

for columns in col_list:
    columns_ = [0,1,2]
    columns_.extend(columns)
    df_temp = df_raw2[columns_]
    df_temp = df_temp.dropna()
    df_temp.columns = columns_base
    df = pd.concat([df, df_temp], axis=0, ignore_index=True)
df.columns = ['C/A', 'UNIT', 'SCP', 'DATE', 'TIME','DESC', 'ENTRIES','EXITS']
df = df.sort_values(by=['C/A', 'UNIT', 'SCP', 'DATE', 'TIME']).reset_index(drop=True)
df.head()

,C/A,UNIT,SCP,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,06-28-14,00:00:00,REGULAR,4679542.0,1591173.0
1,A002,R051,02-00-00,06-28-14,04:00:00,REGULAR,4679583.0,1591180.0
2,A002,R051,02-00-00,06-28-14,08:00:00,REGULAR,4679603.0,1591196.0
3,A002,R051,02-00-00,06-28-14,12:00:00,REGULAR,4679707.0,1591296.0
4,A002,R051,02-00-00,06-28-14,16:00:00,REGULAR,4680003.0,1591342.0


In [78]:
df.groupby(['DESC']).size().sort_values(ascending=False).reset_index(name='Count')

,DESC,Count
0,REGULAR,189026
1,RECOVR AUD,8778
2,DOOR OPEN,6297
3,DOOR CLOSE,3780
4,LOGON,2436
5,LGF-MAN,1693


Looks like there are way more _non-regular_ readings compared to the newer ones.

In [79]:
df.groupby(['C/A', 'UNIT', 'SCP', 'DATE', 'TIME']).size().sort_values(ascending=False).reset_index(name='Count').head()

,C/A,UNIT,SCP,DATE,TIME,Count
0,N332,R219,01-05-01,06-30-14,04:00:00,2
1,C008,R099,00-03-02,07-01-14,04:00:00,2
2,R127,R105,00-03-01,06-30-14,04:00:00,2
3,N010,R126,00-00-03,07-01-14,04:00:00,2
4,N089,R139,00-00-01,07-02-14,04:00:00,2


Looks like there are duplicate values here. Handling it either by averaging it or taking the distinct values will take care of these. Below is an example of what these duplicates may look like.

In [81]:
df[(df['C/A']=='R127') & (df['SCP']=='00-03-01') & (df['DATE']=='06-30-14')]

,C/A,UNIT,SCP,DATE,TIME,DESC,ENTRIES,EXITS
137106,R127,R105,00-03-01,06-30-14,00:00:00,REGULAR,4907579.0,2236569.0
137107,R127,R105,00-03-01,06-30-14,04:00:00,REGULAR,4907688.0,2236578.0
137108,R127,R105,00-03-01,06-30-14,04:00:00,RECOVR AUD,4907688.0,2236578.0
137109,R127,R105,00-03-01,06-30-14,08:00:00,REGULAR,4907783.0,2236630.0
137110,R127,R105,00-03-01,06-30-14,12:00:00,REGULAR,4908117.0,2236835.0
137111,R127,R105,00-03-01,06-30-14,16:00:00,REGULAR,4908479.0,2237025.0
137112,R127,R105,00-03-01,06-30-14,20:00:00,REGULAR,4909174.0,2237276.0


The end result of processing looks like this. In the next notebook we'll take a look at more granular details now that the data is easier to process.

In [83]:
processed_data_folder = os.path.join(os.getcwd(),os.pardir, 'data','processed')
df_processed = pd.read_csv(os.path.join(processed_data_folder, '180721_processed_data.csv'))
df_processed.head()

,UNIT,DATE,INTERVAL,ENTRIES,EXITS,METER_COUNT
0,R051,2018-07-14,00:00:00-04:00:00,575.0,245.0,26
1,R051,2018-07-14,04:00:00-08:00:00,465.0,637.0,26
2,R051,2018-07-14,08:00:00-12:00:00,1258.0,2021.0,26
3,R051,2018-07-14,12:00:00-16:00:00,2715.0,3315.0,26
4,R051,2018-07-14,16:00:00-20:00:00,3278.0,2735.0,26
